<a href="https://colab.research.google.com/github/arahrooh31/UCLA_BE223C/blob/Keane_temp/VGG_DBT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import gc  #debug memory leaks in matplotlib
import csv #read in description files

from google.colab.patches import cv2_imshow
#
# Read Data from google drive
#
from google.colab import drive #for loading gdrive data
from google.colab import files

# install dependencies not included by Colab
# use pip3 to ensure compatibility w/ Google Deep Learning Images 
!pip3 install -q pydicom 
!pip3 install -q tqdm 
!pip3 install -q imgaug
!pip3 install -q pickle5

import pydicom #to read dicom files
from pydicom import dcmread
import pickle5 as pickle; #generic storage of image arra


import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
#show model design parameters with torchsummary
import torchsummary
from torchsummary import summary

### Enable GPU, if present
train_on_gpu = torch.cuda.is_available()
if (train_on_gpu):
    dev=torch.device("cuda")
else:
    print('GPU NOT FOUND!!! USING CPU INSTEAD!!!!!')
#
# Load data from google drive
#

drive.mount('/content/gdrive')

data_dir = '/content/gdrive/My Drive/DBT_DATA/IMG_ARRAYS'
patch_dir = '/content/gdrive/My Drive/DBT_WORKSPACE/TRAINING_PATCHES' 
model_dir = '/content/gdrive/My Drive/BE223C_SPRING_2021/MODEL_SAVE'
#DBT_DATA/TRAINING_DATA/manifest-1605042674814/Breast-Cancer-Screening-DBT'
#png_dir = '/content/gdrive/My Drive/BE223C_SPRING_2021/PNG_IMAGES'
#png_annotation_dir = '/content/gdrive/My Drive/BE223C_SPRING_2021/PNG_ANNOTATION_IMAGES'
#image_save_dir = '/content/gdrive/My Drive/DBT_DATA/IMG_ARRAYS' #matrix img data
#'/content/gdrive/My Drive/BE223C_SPRING_2021/IMAGE_ARRAY'


     |████████████████████████████████| 1.9MB 7.0MB/s 
     |████████████████████████████████| 133kB 6.5MB/s 
GPU NOT FOUND!!! USING CPU INSTEAD!!!!!
Mounted at /content/gdrive


In [2]:
#
# GET FULL LIST OF FILES IN IMAGE ARRAY DIRECTORY
#
use_patch_files = 1
if (use_patch_files == 0):
    raw_files = os.listdir(data_dir)
    print('found #files: ',len(raw_files))
else: #patches broken up into directories
    category_folders = os.listdir(patch_dir)
    #raw_files = os.listdir(data_dir)
    #print('found #files: ',len(raw_files))    

if (0):
    #create fake patches for now
    patch_dict = {}
    for counter,filename in enumerate(raw_files):
        #load full array
        full_filename = os.path.join(data_dir,filename)
        img_data = pickle.load( open( full_filename, "rb" ) )
        patch_data = img_data[0:3,:,:]
        patch_dict[counter]= patch_data
        print(full_filename,np.shape(patch_data))

        if (counter > 0):
            break
        
    

In [3]:
for ii in category_folders:
    print(ii)
    flist = os.listdir(os.path.join(patch_dir,ii))

temp = flist[0].split(sep='_')
print(temp[3])

NORMAL
ACTIONABLE
CANCER
BENIGN
Benign


In [4]:
import os
import pandas as pd
from torchvision.io import read_image

class CustomImageDataset(): #Dataset):
    def __init__(self, img_dir,category=[],file_count=1,transform=None, target_transform=None):
        #self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.category = category
        self.file_count = file_count
        self.transform = transform
        self.target_transform = target_transform


    def __len__(self):
        return self.file_count #len(self.file_list) #99 #len(self.img_labels)

    def __getitem__(self, idx):
        for category_folder in self.category:
            file_list = os.listdir(os.path.join(self.img_dir,category_folder))
            cat_count = 0
            for file_name in file_list:
                cat_count = cat_count + 1
                full_filename = os.path.join(self.img_dir,category_folder, file_name)
                if (cat_count%500 == 0):
                    #print(category_folder,cat_count)
                    pass
                image = pickle.load( open( full_filename, "rb" ) )
                patch_data = image.astype(float) #using patch images
                #patch_data = image[0:3,1200:1444,1200:1444].astype(float)
                image = patch_data
                #get label
                text_tokens = file_name.split(sep='_')
                label_class = text_tokens[3] #get the label token in 4th position 

                #test out numeric label
                if (label_class in 'Normal'):
                    label =0
                elif (label_class in 'Actionable'):
                    label =1
                elif (label_class in 'Benign'):
                    label =2
                else: # (label_class in 'Cancer'):
                    label =3



                self.file_count = self.file_count + 1          

        #for ii in self.file_list:
        #    full_filename = os.path.join(self.img_dir,ii) #self.img_dir# = os.path.join(self.img_dir) self.img_labels.iloc[idx, 0])
        #    image = pickle.load( open( full_filename, "rb" ) )
        #    patch_data = image.astype(float) #using patch images
        #    #patch_data = image[0:3,1200:1444,1200:1444].astype(float)
        #    image = patch_data
        #    label = 1

            #read_image(img_path)
            #label = self.img_labels.iloc[idx, 1]
            if self.transform:
                image = self.transform(image)
            if self.target_transform:
                label = self.target_transform(label)
            sample = {"image": image, "label": label}
            return sample

In [7]:
from torch.utils.data import DataLoader

#
count =0
# Setup the custom dataset
for ii in category_folders:
        nfiles = os.listdir(os.path.join(patch_dir,ii))
        count = count + len(nfiles)


#load up with the pre-sized patch images
training_data = CustomImageDataset(img_dir=patch_dir,
                                   category = category_folders, 
                                   file_count=count,transform=None, 
                                   target_transform=None)

#
# TEST OUT SPLITTING DATASETS INTO TRAIN/TEST
#
train_size = int(0.8 * len(training_data))
test_size = len(training_data) - train_size
train_subset, test_subset = torch.utils.data.random_split(training_data, [train_size, test_size])

bsize = 8
#training_data = CustomImageDataset( annotations_file='', img_dir=data_dir, file_list=raw_files,transform=None, target_transform=None)
if (train_on_gpu):
    dataloader_training = DataLoader(train_subset, batch_size=bsize,shuffle=True, num_workers=4) #only 2 workers for Colab CPU
    dataloader_test = DataLoader(test_subset, batch_size=bsize,shuffle=True, num_workers=4) #only 2 workers for Colab CPU
else:
    dataloader_training = DataLoader(test_subset, batch_size=bsize,shuffle=True, num_workers=2) #only 2 workers for Colab CPU
    dataloader_test = DataLoader(test_subset, batch_size=bsize,shuffle=True, num_workers=2) #only 2 workers for Colab CPU


############################################
if (0): #disable while testing above, but this works
    bsize = 8
    #training_data = CustomImageDataset( annotations_file='', img_dir=data_dir, file_list=raw_files,transform=None, target_transform=None)
    if (train_on_gpu):
        dataloader = DataLoader(training_data, batch_size=bsize,shuffle=True, num_workers=4) #only 2 workers for Colab CPU
    else:
        dataloader = DataLoader(training_data, batch_size=bsize,shuffle=True, num_workers=2) #only 2 workers for Colab CPU


In [6]:
### DEBUG
#for ii in training_data:
#    print(ii)
if (0):
    for i, data in enumerate(dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        print(type(data['image']))

In [8]:
from torch import nn

class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        self.flatten = nn.Flatten()
        self.vgg16_stack = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),  
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),         
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),        
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),            
            nn.ReLU(inplace=True),         
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),         
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Linear(7, 4096), #in should match 512x512 above
            nn.Linear(4096,4))
        #nn.Sequential(nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)




    def forward(self, x):
        self.flatten = nn.Flatten()
        #x = self.flatten(x)
        logits = self.vgg16_stack(x)
        return logits



model_vgg16 = VGG16() #.to(device)
model_vgg16 = model_vgg16.float()
print(model_vgg16)

VGG16(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (vgg16_stack): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (13): Conv2d(256, 512, kernel_size=(3, 3),

In [8]:
#clear GPU cache if needed

try_to_clear = 0
if (try_to_clear == 1):
    del model_vgg16
    mem_alloc = torch.cuda.memory_allocated(dev)
    gc.collect()
    torch.cuda.empty_cache() #   clear_cache

    print('memory allocated is ', mem_alloc)
else:
    print('SKIP GPU MEM CLEAR---')

SKIP GPU MEM CLEAR---


In [9]:
#Show summary of model setup and move model to the GPU
 #train_on_gpu = torch.cuda.is_available()

if (train_on_gpu == 1):
    #dev=torch.device("cuda") 
    model_vgg16.to(dev)
    summary(model_vgg16,(3,244,244), batch_size = bsize, device='cuda')

In [10]:
#
# LOSS FUNCTION SETUP
#
import torch.optim as optim
from torchsummary import summary

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_vgg16.parameters(), lr=0.001, momentum=0.9)
#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

L1loss = nn.L1Loss()

model_vgg16.parameters
#summary(model_vgg16, (3, 224, 224))

<bound method Module.parameters of VGG16(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (vgg16_stack): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (13): C

In [ ]:
### TRAIN THE NETWORK
from torch import FloatTensor
from torch import tensor


for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    #for i, data in enumerate(dataloader, 0):  #works for general setup!!
    for i, data in enumerate(dataloader_training, 0):
        # get the inputs; data is a list of [inputs, labels]
        #inputs, labels = data
        inputs = data['image'].type(FloatTensor)
        labels = data['label'] #.type(FloatTensor)

        if (train_on_gpu):
            inputs, labels = inputs.to(dev), labels.to(dev)



        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model_vgg16(inputs) #.permute(0, 1, 2, 3))
        #print(outputs.size())
        #print(labels.size())

        #print('outputs type is ', outputs.size())
        #print(outputs)
        
        ### L1 loss complains about outputs columns being out of order, permute
        ### outputs to match up
        use_L1 = 1
        if (use_L1==1):
            outputs= outputs.permute(3,1,2,0)
            loss = L1loss(outputs,labels)
        else:
            loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        # print statistics
        if (i % 100 == 0):
            running_loss += loss.item()
            print('loop,epoch, loss, total running loss = ',i,epoch, loss.item(), running_loss/100)
            running_loss = 0.0
        #if i % 100 == 0:    # print every 10 mini-batches
        #    print('[%d, %5d] loss: %.3f' %
        #          (epoch + 1, i + 1, running_loss/100))
        #    running_loss = 0.0

torch.save(model_vgg16.state_dict(),os.path.join(model_dir,'model_final'))
print('Finished Training')

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([4, 512, 7, 8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


loop,epoch, loss, total running loss =  0 0 0.29905837774276733 0.0029905837774276732


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([7])) that is different to the input size (torch.Size([4, 512, 7, 7])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


loop,epoch, loss, total running loss =  0 1 0.03274314105510712 0.00032743141055107116


In [12]:
#model_vgg16.state_dict()

torch.save(model_vgg16.state_dict(),os.path.join(model_dir,'model_final'))
#outputs= outputs.permute(3,1,2,0)
#outputs.view()

In [13]:
import torch # PyTorch 
from torchvision import datasets # Datasets module 
import torchvision.transforms as transforms # Image Transforms 
from torch.utils.data.sampler import SubsetRandomSampler # Sampler 
!pip3 -q install torchlayers
import torchlayers as tl

In [14]:

 
'''VGG11/13/16/19 in Pytorch.'''
import torch
import torch.nn as nn


cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(512, 10)

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)


def test():
    net = VGG('VGG16')
    x = torch.randn(2,3,32,32)
    y = net(x)
    print(y.size())
    print(net)

test()


torch.Size([2, 10])
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (

In [15]:

 
'''Train CIFAR10 with PyTorch.'''
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

from models import *
from utils import progress_bar


parser = argparse.ArgumentParser(description='PyTorch CIFAR10 Training')
parser.add_argument('--lr', default=0.1, type=float, help='learning rate')
parser.add_argument('--resume', '-r', action='store_true',
                    help='resume from checkpoint')
args = parser.parse_args()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')
# net = VGG('VGG19')
# net = ResNet18()
# net = PreActResNet18()
# net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
# net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)
# net = EfficientNetB0()
# net = RegNetX_200MF()
net = SimpleDLA()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

if args.resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/ckpt.pth')
    net.load_state_dict(checkpoint['net'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=args.lr,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)


# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))


def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc


for epoch in range(start_epoch, start_epoch+200):
    train(epoch)
    test(epoch)
    scheduler.step()


ModuleNotFoundError: ignored